In [ ]:
# 来源于工具文件
import torch
import torch.nn as nn
import math
import yaml
from pathlib import Path
import contextlib
from copy import deepcopy

class Expand(nn.Module):
    def __init__(self, gain=2):
        super().__init__()
        self.gain = gain

    def forward(self, x):
        b, c, h, w = x.size()  # assert C / s ** 2 == 0, 'Indivisible gain'
        s = self.gain
        x = x.view(b, s, s, c // s ** 2, h, w)  # x(1,2,2,16,80,80)
        x = x.permute(0, 3, 4, 1, 5, 2).contiguous()  # x(1,16,80,2,80,2)
        return x.view(b, c // s ** 2, h * s, w * s)  # x(1,16,160,160)
class Classify(nn.Module):
    # Classification head, i.e. x(b,c1,20,20) to x(b,c2)
    def __init__(self, c1, c2, k=1, s=1, p=None, g=1):  # ch_in, ch_out, kernel, stride, padding, groups
        super().__init__()
        c_ = 1280  # efficientnet_b0 size
        self.conv = Conv(c1, c_, k, s, autopad(k, p), g)
        self.pool = nn.AdaptiveAvgPool2d(1)  # to x(b,c_,1,1)
        self.drop = nn.Dropout(p=0.0, inplace=True)
        self.linear = nn.Linear(c_, c2)  # to x(b,c2)

    def forward(self, x):
        if isinstance(x, list):
            x = torch.cat(x, 1)
        return self.linear(self.drop(self.pool(self.conv(x)).flatten(1)))

class Contract(nn.Module):
    # Contract width-height into channels, i.e. x(1,64,80,80) to x(1,256,40,40)
    def __init__(self, gain=2):
        super().__init__()
        self.gain = gain

    def forward(self, x):
        b, c, h, w = x.size()  # assert (h / s == 0) and (W / s == 0), 'Indivisible gain'
        s = self.gain
        x = x.view(b, c, h // s, s, w // s, s)  # x(1,64,40,2,40,2)
        x = x.permute(0, 3, 5, 1, 2, 4).contiguous()  # x(1,2,2,64,40,40)
        return x.view(b, c * s * s, h // s, w // s)  # x(1,256,40,40)

class Concat(nn.Module):
    # Concatenate a list of tensors along dimension
    def __init__(self, dimension=1):
        super().__init__()
        self.d = dimension

    def forward(self, x):
        return torch.cat(x, self.d)
def make_divisible(x, divisor):
    # Returns nearest x divisible by divisor
    if isinstance(divisor, torch.Tensor):
        divisor = int(divisor.max())  # to int
    return math.ceil(x / divisor) * divisor


def autopad(k, p=None):  # kernel, padding
    # Pad to 'same'
    if p is None:
        p = k // 2 if isinstance(k, int) else [x // 2 for x in k]  # auto-pad
    return p

class Conv(nn.Module):
    # Standard convolution
    def __init__(self, c1, c2, k=1, s=1, p=None, g=1, act=True):  # ch_in, ch_out, kernel, stride, padding, groups
        super().__init__()
        self.conv = nn.Conv2d(c1, c2, k, s, autopad(k, p), groups=g, bias=False)
        self.bn = nn.BatchNorm2d(c2)
        self.act = nn.SiLU() if act is True else (act if isinstance(act, nn.Module) else nn.Identity())

    def forward(self, x):
        return self.act(self.bn(self.conv(x)))

    def forward_fuse(self, x):
        return self.act(self.conv(x))

class Bottleneck(nn.Module):
    # Standard bottleneck
    def __init__(self, c1, c2, shortcut=True, g=1, e=0.5):  # ch_in, ch_out, shortcut, groups, expansion
        super().__init__()
        c_ = int(c2 * e)  # hidden channels
        self.cv1 = Conv(c1, c_, 1, 1)
        self.cv2 = Conv(c_, c2, 3, 1, g=g)
        self.add = shortcut and c1 == c2

    def forward(self, x):
        return x + self.cv2(self.cv1(x)) if self.add else self.cv2(self.cv1(x))
    
class C3(nn.Module):
    # CSP Bottleneck with 3 convolutions
    def __init__(self, c1, c2, n=1, shortcut=True, g=1, e=0.5):  # ch_in, ch_out, number, shortcut, groups, expansion
        super().__init__()
        c_ = int(c2 * e)  # hidden channels
        self.cv1 = Conv(c1, c_, 1, 1)
        self.cv2 = Conv(c1, c_, 1, 1)
        self.cv3 = Conv(2 * c_, c2, 1)  # optional act=FReLU(c2)
        self.m = nn.Sequential(*(Bottleneck(c_, c_, shortcut, g, e=1.0) for _ in range(n)))
    def forward(self, x):
        return self.cv3(torch.cat((self.m(self.cv1(x)), self.cv2(x)), 1))
    

###### 使用conv3x3替换掉上面代码块里的Conv里面的普通卷积#####
------------------------
```python3
class dwconv(nn.Module):
    def __init__(self,in_ch,out_ch,ks=3,st=1,pd=1,gp=1,dn=1):
        super(dwconv, self).__init__()
        # 也相当于分组为1的分组卷积
        self.depth_conv = nn.Conv2d(in_channels=in_ch,
                                    out_channels=in_ch,
                                    kernel_size=ks,
                                    stride=st,
                                    padding=pd,
                                    groups=in_ch)
        self.point_conv = nn.Conv2d(in_channels=in_ch,
                                    out_channels=out_ch,
                                    kernel_size=1,
                                    stride=1,
                                    padding=0,
                                    groups=gp,dilation=dn)
    def forward(self,input):
        out = self.depth_conv(input)
        out = self.point_conv(out)
        return out


def conv3x3(in_planes: int, out_planes: int, stride: int = 1, groups: int = 1, dilation: int = 1) -> nn.Conv2d:
    """3x3 convolution with padding"""
    return  dwconv(in_planes,out_planes,ks=3,st=stride,pd=dilation,gp=groups,dn=dilation)
```
------------------------

In [ ]:
# 构建网络代码
import torch.nn as nn
from yolov5_cls_utils import C3,Conv,Classify,make_divisible
import pprint
import torch
class net_cfg:
    def __init__(self,
                 num_classes1:int=10,
                 num_classes2:int=10,
                 num_classes3:int=10):
        self.num_cls_1 = num_classes1
        self.num_cls_2 = num_classes2
        self.num_cls_3 = num_classes3
        
    def yolov5_s(self):
        """
        0    1      3520  models.common.Conv         [3, 32, 6, 2, 2]
        1    1     18560  models.common.Conv         [32, 64, 3, 2]
        2    1     18816  models.common.C3           [64, 64, 1]
        3    1     73984  models.common.Conv         [64, 128, 3, 2]
        4    2    115712  models.common.C3           [128, 128, 2]
        5    1    295424  models.common.Conv         [128, 256, 3, 2]
        6    3    625152  models.common.C3           [256, 256, 3]
        7    1   1180672  models.common.Conv         [256, 512, 3, 2]
        8    1   1182720  models.common.C3           [512, 512, 1]
        """
        cfg = {
                "feature_map":[
                                [[1],[3,32,6,2,2]],     # conv  0
                                [[1],[32,64,3,2]],      # conv  1
                                [[3],[64,64,1]],        # c3    2
                                [[1],[64,128,3,2]],     # conv  3
                                [[6],[128,128,2]],      # c3    4
                                [[1],[128,256,3,2]],    # conv  5
                                [[9],[256,256,3]],      # c3    6
                                [[1],[256,512,3,2]],    # conv  7
                                [[3],[512,512,1]],      # c3    8
                                ],
               "classify":{"1":[512,self.num_cls_1],
                           "2":[512,self.num_cls_2],
                           "3":[512,self.num_cls_3]}
               }
        return cfg
    def yolov5_n(self):
        """
        0    1      1760  models.common.Conv         [3, 16, 6, 2, 2]
        1    1      4672  models.common.Conv         [16, 32, 3, 2]
        2    1      4800  models.common.C3           [32, 32, 1]
        3    1     18560  models.common.Conv         [32, 64, 3, 2]
        4    2     29184  models.common.C3           [64, 64, 2]
        5    1     73984  models.common.Conv         [64, 128, 3, 2]
        6    3    156928  models.common.C3           [128, 128, 3]
        7    1    295424  models.common.Conv         [128, 256, 3, 2]
        8    1    296448  models.common.C3           [256, 256, 1]
        """
        cfg = {
                "feature_map":[
                                [[1],[3,16,6,2,2]],     # conv  0
                                [[1],[16,32,3,2]],      # conv  1
                                [[1],[32,32,1]],        # c3    2
                                [[1],[32,64,3,2]],      # conv  3
                                [[2],[64,64,2]],        # c3    4
                                [[1],[64,128,3,2]],     # conv  5
                                [[3],[128,128,3]],      # c3    6
                                [[1],[128,256,3,2]],    # conv  7
                                [[1],[256,256,1]],      # c3    8
                                ],
               "classify":{"1":[256,self.num_cls_1],
                           "2":[256,self.num_cls_2],
                           "3":[256,self.num_cls_3]}
               }
        return cfg
    def yolov5_l(self):  # 弃用
        """
        0   1      7040  models.common.Conv    [3, 64, 6, 2, 2]
        1   1     73984  models.common.Conv    [64, 128, 3, 2]
        2   3    156928  models.common.C3      [128, 128, 3]
        3   1    295424  models.common.Conv    [128, 256, 3, 2]
        4   6   1118208  models.common.C3      [256, 256, 6]
        5   1   1180672  models.common.Conv    [256, 512, 3, 2]
        6   9   6433792  models.common.C3      [512, 512, 9]
        7   1   4720640  models.common.Conv    [512, 1024, 3, 2]
        8   3   9971712  models.common.C3      [1024, 1024, 3]
        """
        cfg = {
                "feature_map":[
                                [[1],[3,64,6,2,2]],     # conv  0
                                [[1],[64,128,3,2]],      # conv  1
                                [[1],[128,128,3]],        # c3    2
                                [[1],[128,256,3,2]],      # conv  3
                                [[2],[256,256,6]],        # c3    4
                                [[1],[256,512,3,2]],     # conv  5
                                [[3],[512,512,9]],      # c3    6
                                [[1],[512,1024,3,2]],    # conv  7
                                [[1],[1024,1024,3]],      # c3    8
                                ],
               "classify":{"1":[1024,self.num_cls_1],
                           "2":[1024,self.num_cls_2],
                           "3":[1024,self.num_cls_3]}
               }
        return cfg
    def yolov5_m(self): # 弃用
        """
        0    1      5280  models.common.Conv     [3, 48, 6, 2, 2]
        1    1     41664  models.common.Conv     [48, 96, 3, 2]
        2    2     65280  models.common.C3       [96, 96, 2]
        3    1    166272  models.common.Conv     [96, 192, 3, 2]
        4    4    444672  models.common.C3       [192, 192, 4]
        5    1    664320  models.common.Conv     [192, 384, 3, 2]
        6    6   2512896  models.common.C3       [384, 384, 6]
        7    1   2655744  models.common.Conv     [384, 768, 3, 2]
        8    2   4134912  models.common.C3       [768, 768, 2]
        """
        cfg = {
                "feature_map":[
                                [[1],[3,48,6,2,2]],     # conv  0
                                [[1],[48,96,3,2]],      # conv  1
                                [[1],[96,96,2]],        # c3    2
                                [[1],[96,192,3,2]],      # conv  3
                                [[2],[192,192,4]],        # c3    4
                                [[1],[192,384,3,2]],     # conv  5
                                [[3],[384,384,6]],      # c3    6
                                [[1],[384,768,3,2]],    # conv  7
                                [[1],[768,768,2]],      # c3    8
                                ],
               "classify":{"1":[768,self.num_cls_1],
                           "2":[768,self.num_cls_2],
                           "3":[768,self.num_cls_3]}
               }
        return cfg
    
class yolov5_cls_cn(nn.Module):
    def __init__(self,
                 cfg:dict,
                 flg:str='s'
                 ):
        """
            conv使用的正常卷积
        """
        super(yolov5_cls_cn, self).__init__()
        self.cfg = cfg
        self.flg = flg
        if self.flg =='n':
            self.feature_ = self._build_feature_map_1()
            self.cls_1 = self._build_cls_head_1('1')
            self.cls_2 = self._build_cls_head_1('2')
            self.cls_3 = self._build_cls_head_1('3')
        elif self.flg=='s':
            self.feature_ = self._build_feature_map_2()
            self.cls_1 = self._build_cls_head_2('1')
            self.cls_2 = self._build_cls_head_2('2')
            self.cls_3 = self._build_cls_head_2('3')
        else:
            raise RuntimeError
    def _build_feature_map_1(self):
        feature_map = nn.ModuleList()
        # feature_map = []
        feature_map_cfg = self.cfg['feature_map']
        
        feature_map.append(Conv(*feature_map_cfg[0][1]))    
        feature_map.append(Conv(*feature_map_cfg[1][1])) 
        feature_map.extend([C3(*feature_map_cfg[2][1])]*feature_map_cfg[2][0][0]) 
        feature_map.append(Conv(*feature_map_cfg[3][1])) 
        feature_map.extend([C3(*feature_map_cfg[4][1])]*feature_map_cfg[4][0][0]) 
        feature_map.append(Conv(*feature_map_cfg[5][1])) 
        feature_map.extend([C3(*feature_map_cfg[6][1])]*feature_map_cfg[6][0][0]) 
        feature_map.append(Conv(*feature_map_cfg[7][1])) 
        feature_map.extend([C3(*feature_map_cfg[8][1])]*feature_map_cfg[8][0][0]) 
      
        return nn.Sequential(*feature_map)
    def _build_cls_head_1(self,indx):
        classify = nn.Sequential()
        classify_cfg = self.cfg['classify']
        classify.add_module(f'cls_{indx}',Classify(*classify_cfg[indx]))
        return classify
    
    def _build_feature_map_2(self):
        feature_map = nn.ModuleList()
        feature_map_cfg = self.cfg['feature_map']
        feature_map.append(Conv(*feature_map_cfg[0][1]))    
        feature_map.append(Conv(*feature_map_cfg[1][1])) 
        feature_map.extend([C3(*feature_map_cfg[2][1])]*feature_map_cfg[2][0][0]) 
        feature_map.append(Conv(*feature_map_cfg[3][1])) 
        feature_map.extend([C3(*feature_map_cfg[4][1])]*feature_map_cfg[4][0][0]) 
        feature_map.append(Conv(*feature_map_cfg[5][1])) 
        feature_map.extend([C3(*feature_map_cfg[6][1])]*feature_map_cfg[6][0][0]) 
        feature_map.append(Conv(*feature_map_cfg[7][1])) 
        last_n = feature_map_cfg[8][0][0]
        if last_n>1:
            feature_map.extend([C3(*feature_map_cfg[8][1])]*(last_n-1)) 
        return nn.Sequential(*feature_map)
    
    def _build_cls_head_2(self,indx):
        classify = nn.Sequential()
        feature_map_cfg = self.cfg['feature_map']
        classify_cfg = self.cfg['classify']
        classify.add_module(f'cls_C3_{indx}',C3(*feature_map_cfg[8][1])) 
        classify.add_module(f'cls_fc_{indx}',Classify(*classify_cfg[indx]))
        return classify

    def forward(self,x):
        x = self.feature_(x)
        out1 = self.cls_1(x)
        out2 = self.cls_2(x)
        out3 = self.cls_3(x)
        return [out1,out2,out3]

class yolov5_cls_s(nn.Module):
    def __init__(self,
                 cfg:dict,
                 flg:str='s'
                 ):
        super(yolov5_cls_s, self).__init__()
        self.cfg = cfg
        self.flg = flg
        self.feature_ = self._build_feature_map_1()
        self.cls_1 = self._build_cls_head_1('1')
        self.cls_2 = self._build_cls_head_1('2')
        self.cls_3 = self._build_cls_head_1('3')
    def _build_feature_map_1(self):
        feature_map = nn.ModuleList()
        # feature_map = []
        feature_map_cfg = self.cfg['feature_map']
        
        feature_map.append(Conv(*feature_map_cfg[0][1]))    
        feature_map.append(Conv(*feature_map_cfg[1][1])) 
        # print(feature_map_cfg[2][1],feature_map_cfg[2][0])
        feature_map.extend([C3(*feature_map_cfg[2][1])]*feature_map_cfg[2][0][0]) 
        feature_map.append(Conv(*feature_map_cfg[3][1])) 
        feature_map.extend([C3(*feature_map_cfg[4][1])]*feature_map_cfg[4][0][0]) 
        feature_map.append(Conv(*feature_map_cfg[5][1])) 
        feature_map.extend([C3(*feature_map_cfg[6][1])]*feature_map_cfg[6][0][0]) 
        feature_map.append(Conv(*feature_map_cfg[7][1])) 
        feature_map.extend([C3(*feature_map_cfg[8][1])]*feature_map_cfg[8][0][0]) 
      
        return nn.Sequential(*feature_map)
    def _build_cls_head_1(self,indx):
        # classify = nn.ModuleList()
        classify = nn.Sequential()
        classify_cfg = self.cfg['classify']
        classify.add_module(f'cls_{indx}',Classify(*classify_cfg[indx]))
        return classify

    def forward(self,x):
        x = self.feature_(x)
        out1 = self.cls_1(x)
        out2 = self.cls_2(x)
        out3 = self.cls_3(x)
        return [out1,out2,out3]
    
if __name__ == "__main__":
    cfg = net_cfg(num_classes1=4,num_classes2=3,num_classes3=2).yolov5_s()
    pprint.pprint(cfg)
    pth = "yolocls-n.pt"
    #! 0
    # model = yolov5_cls_s(cfg,'n')
    # data = torch.randn(1,3,224,224)
    # pprint.pprint(model(data))
    # torch.save(model,pth)    
    # import os
    # print(f"{pth}    {os.stat(pth).st_size/1024/1024:.4f} M")
    #! 1
    model = yolov5_cls_cn(cfg,'s')
    data = torch.randn(1,3,224,224)
    pprint.pprint(model(data))
    torch.save(model,pth)    
    import os
    print(f"{pth}    {os.stat(pth).st_size/1024/1024:.4f} M")
    
    #! 2 
    # cfg = net_cfg(num_classes1=4,num_classes2=3,num_classes3=2).yolov5_m()
    # pprint.pprint(cfg)
    # pth = "yolocls-n.pt"
    # model = yolov5_cls_cn(cfg,'s')
    # data = torch.randn(1,3,224,224)
    # pprint.pprint(model(data))
    # torch.save(model,pth)    
    # import os
    # print(f"{pth}    {os.stat(pth).st_size/1024/1024:.4f} M")

# 需求
- 把网络中的普通卷积替换成我上面的那种矩阵分解卷积。主要是改Class Conv类中ksize不为1的。分解卷积在第二个代码块里
- 主要针对 yolov5-s  yolov5-m yolov5-n 不用管yolov5-l。
尽快做